In [1]:
#정보요청 라이브러리 import
import requests

#크롤링 라이브러리 import
from bs4 import BeautifulSoup 

#셀레늄 라이브러리 불러오기
from selenium import webdriver

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

#셀레늄에서 html문서가 생성될 때까지 대기하는 함수 불러오기
from selenium.webdriver.support.ui import WebDriverWait

#셀레늄에서 조건을 걸어줄 수 있는 함수 불러오기
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import pandas as pd
import re
import time
import random

In [2]:
#User-Agent 지정
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}

In [3]:
# URL 설정 
URL = 'https://hogangnono.com/'

In [4]:
#드라이버 객체 생성
browser = webdriver.Chrome(service = Service(ChromeDriverManager().install()))
browser.get(URL)

### 로그인

In [5]:
login__ = '//*[@id="container"]/div[3]/a'

In [7]:
browser.find_element(by = By.XPATH, value = login__).click()

In [ ]:
kakao_login__ = '//*[@id="react-root"]/div/div[2]/div[2]/div/div/div/a[2]'

In [ ]:
browser.find_element(by = By.XPATH, value = kakao_login__).click()

### 아파트 명칭 검색

In [23]:
apt_list = ['위례그린파크 푸르지오']
search_key = '//*[@id="container"]/div[2]/fieldset/div/div[1]/input'

In [24]:
browser.find_element(by = By.XPATH, value = search_key).send_keys(apt_list[0]) #아파트 이름 누르기
browser.find_element(by = By.XPATH, value = search_key).send_keys('\n') # enter누르기

### 검색결과 내 아파트 선택 

In [25]:
apt_search__ = '//*[@id="container"]/div[4]/div/div/div[1]/div/ul/li[{apt_search_num}]/a'.format(apt_search_num = 1) # 임의로 1 설정 
browser.find_element(by = By.XPATH, value = apt_search__).click()

### 리뷰 버튼 클릭 

In [122]:
# '//*[@id="container"]/div[4]/div/div/fieldset/div[4]/ul/li[2]/button'
apt_review_button = '//*[@id="container"]/div[4]/div/div/fieldset/div[4]/ul/li[3]/a'
review_count = browser.find_element(by = By.XPATH, value = apt_review_button).text
browser.find_element(by = By.XPATH, value = apt_review_button).click()

print(review_count)

322


### 아이디 

In [18]:
nickname__ = '//*[@id="container"]/div[4]/div[2]/div[2]/div/div[1]/div/div[3]/div[1]/div[1]/span[1]'
hash_id__ = '//*[@id="container"]/div[4]/div[2]/div[2]/div/div[1]/div/div[3]/div[1]/div[1]/span[2]'
review__ = '//*[@id="container"]/div[4]/div[2]/div[2]/div/div[1]/div/div[3]/div[1]/div[2]'

nickname = browser.find_element(by = By.XPATH, value = nickname__).text
hash_id = browser.find_element(by = By.XPATH, value = hash_id__).text
review = browser.find_element(by = By.XPATH, value = review__).text

print(nickname)
print(hash_id)
print(review)

In [157]:
def scroll(modal):
    try:        
        # 스크롤 높이 받아오기
        last_height = browser.execute_script("return arguments[0].scrollHeight", modal)
        while True:
            pause_time = random.uniform(0.5, 0.8)
            # 최하단까지 스크롤
            browser.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", modal)
            # 페이지 로딩 대기
            time.sleep(pause_time)
            # 무한 스크롤 동작을 위해 살짝 위로 스크롤
            browser.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight-50);", modal)
            time.sleep(pause_time)
            # 스크롤 높이 새롭게 받아오기
            new_height = browser.execute_script("return arguments[0].scrollHeight", modal)
            try:
                # '더보기' 버튼 있을 경우 클릭
                all_review_button = browser.find_element(by = By.XPATH, value = more_see).click()
            except:
                # 스크롤 완료 경우
                if new_height == last_height:
                    print("스크롤 완료")
                    break
                last_height = new_height
    except: 
        pass

In [158]:
modal = WebDriverWait(browser, 2).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='css-1poz24i scroll-content']")))
scroll(modal)

스크롤 완료


### 최종 리뷰 크롤링

In [160]:
from tqdm import tqdm

total_review_dict = {}
pause_time = random.uniform(0.5, 0.8)

for i in tqdm(range(int(review_count))):
    review_dict = {}
    try: 
        try:
            nickname__  = '//*[@id="container"]/div[4]/div[2]/div[2]/div/div[1]/div/div[{num}]/div[1]/div[1]/span[1]'.format(num = i+3)
            hash_id__   = '//*[@id="container"]/div[4]/div[2]/div[2]/div/div[1]/div/div[{num}]/div[1]/div[1]/span[2]'.format(num = i+3)
            review__    = '//*[@id="container"]/div[4]/div[2]/div[2]/div/div[1]/div/div[{num}]/div[1]/div[2]'.format(num = i+3)

            nickname = browser.find_element(by = By.XPATH, value = nickname__).text
            hash_id = browser.find_element(by = By.XPATH, value = hash_id__).text
            review = browser.find_element(by = By.XPATH, value = review__).text

            if review[-3:] == '더보기':
                browser.find_element(by = By.XPATH, value = review__+'/a').click()
                time.sleep(pause_time)
                review = browser.find_element(by = By.XPATH, value = review__).text

            review_dict['nickname'] = nickname
            review_dict['hash_id'] = hash_id
            review_dict['review'] = review


        except:
            nickname__  = '//*[@id="container"]/div[4]/div[2]/div[2]/div[1]/div[1]/div/div[{num}]/div[1]/div[1]/span[1]'.format(num = i+3)
            hash_id__   = '//*[@id="container"]/div[4]/div[2]/div[2]/div[1]/div[1]/div/div[{num}]/div[1]/div[1]/span[2]'.format(num = i+3)
            review__    = '//*[@id="container"]/div[4]/div[2]/div[2]/div[1]/div[1]/div/div[{num}]/div[1]/div[2]'.format(num = i+3)


            nickname = browser.find_element(by = By.XPATH, value = nickname__).text
            hash_id = browser.find_element(by = By.XPATH, value = hash_id__).text
            review = browser.find_element(by = By.XPATH, value = review__).text

            if review[-3:] == '더보기':
                time.sleep(pause_time)
                browser.find_element(by = By.XPATH, value = str(review__)+'/a').click()
                time.sleep(pause_time)
                review = browser.find_element(by = By.XPATH, value = review__).text


            review_dict['nickname'] = nickname
            review_dict['hash_id'] = hash_id
            review_dict['review'] = review


        total_review_dict['review_'+str(i)] = review_dict
    except: 
        pass

100%|█████████▉| 321/322 [03:04<00:00,  1.74it/s]


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="container"]/div[4]/div[2]/div[2]/div[1]/div[1]/div/div[324]/div[1]/div[1]/span[1]"}
  (Session info: chrome=106.0.5249.119)
Stacktrace:
Backtrace:
	Ordinal0 [0x00B51ED3+2236115]
	Ordinal0 [0x00AE92F1+1807089]
	Ordinal0 [0x009F66FD+812797]
	Ordinal0 [0x00A255DF+1005023]
	Ordinal0 [0x00A257CB+1005515]
	Ordinal0 [0x00A57632+1209906]
	Ordinal0 [0x00A41AD4+1120980]
	Ordinal0 [0x00A559E2+1202658]
	Ordinal0 [0x00A418A6+1120422]
	Ordinal0 [0x00A1A73D+960317]
	Ordinal0 [0x00A1B71F+964383]
	GetHandleVerifier [0x00DFE7E2+2743074]
	GetHandleVerifier [0x00DF08D4+2685972]
	GetHandleVerifier [0x00BE2BAA+532202]
	GetHandleVerifier [0x00BE1990+527568]
	Ordinal0 [0x00AF080C+1837068]
	Ordinal0 [0x00AF4CD8+1854680]
	Ordinal0 [0x00AF4DC5+1854917]
	Ordinal0 [0x00AFED64+1895780]
	BaseThreadInitThunk [0x76BB6739+25]
	RtlGetFullPathName_UEx [0x77C98FD2+1218]
	RtlGetFullPathName_UEx [0x77C98F9D+116

In [161]:
total_review_dict['review_1']

{'nickname': '부가비',
 'hash_id': '2JSW5dd',
 'review': '궁금한 점이 있습니다\n\n단지가 하남과 성남으로 나뉜다고 들었는데 \n제가 아는게 맞는걸까요? 그러면 초등학교나\n중학교도 나뉘는 건가요?\n \n정말 좋은 단지 같아서\n살고 싶은 마음에 여쭤봅니다'}

### json 저장

In [162]:
import os
import json

file_path = os.getcwd()
file_name = '\\apt_review_dict'

with open(file_path+file_name, 'w') as outfile:
    json.dump(total_review_dict, outfile, indent=4)

### json 열기

In [163]:
import os
import json

file_path = os.getcwd()
file_name = '\\apt_review_dict'

with open(file_path+file_name, "r") as json_file:
    json_data = json.load(json_file)
    print(json_data)
    print("")

{'review_0': {'nickname': '하이하이', 'hash_id': 'nXL9696', 'review': '좋아요!'}, 'review_1': {'nickname': '부가비', 'hash_id': '2JSW5dd', 'review': '궁금한 점이 있습니다\n\n단지가 하남과 성남으로 나뉜다고 들었는데 \n제가 아는게 맞는걸까요? 그러면 초등학교나\n중학교도 나뉘는 건가요?\n \n정말 좋은 단지 같아서\n살고 싶은 마음에 여쭤봅니다'}, 'review_2': {'nickname': 'Greenlight', 'hash_id': '2d6Fa3b', 'review': '아파트 부근에 위치한 개천에서 느낄수 있는\n평화롭고 아름다운 늦여름의 정취.'}, 'review_3': {'nickname': 'Skwksidod', 'hash_id': 'aGjw543', 'review': '버스 타고 들어오는 데 다소 시간은 걸리지만, 큰 차도와 떨어져있어 공기가 맑고 조용한 것이 아주 마음에 듭니다. \n퇴근하고 오면 퇴근한 느낌이 확실히 들어요.'}, 'review_4': {'nickname': 'dream', 'hash_id': '52bH4ac', 'review': '방3개 구조에서 2주방을 방으로 변경하고 싶은데\n복잡할까요?\n경험 있으신분들 조언 부탁드립니다'}, 'review_5': {'nickname': '0726383', 'hash_id': 'mtDcc90', 'review': '산책 하기도 좋고 공기도 맑고 너무 살기 좋아요'}, 'review_6': {'nickname': 'Ruru', 'hash_id': 'ugdD1c5', 'review': '산이 옆에 있어 공기좋고 조용해요\n등산 하기도 좋아요 조경도 넘 잘되어있구요'}, 'review_7': {'nickname': '2**9', 'hash_id': 'tC8yfd0', 'review': '매일 등산할 수 있고 아침에 새소리가 들려요 숲속에 사는거같아 너무 좋아요'}, 'review_8': 